In [17]:
end_of_string = '\u1026' # character at the end of each string

In [170]:
class Trie:
    class Trie_Node:
        def __init__(self, letter, parent, children = None, link = None):
            self.letter = letter
            self.parent = parent
            
            if link is None:
                self.link = self
            else:
                self.link = link
            
            if children is None:
                self.children = dict()
            else:
                self.children = children
                
        def add_child(self, child):

            self.children[child.letter] = child
            
        def is_child(self,letter):
            return letter in self.children.keys()
        
        def get_child(self, letter):
            return self.children[letter]
        
            
    def __init__(self, text):
        
        self.root = self.Trie_Node(None, None)
        
        text = self.check_text(text)
        
        longest = self.Trie_Node(text[0], self.root, link=self.root)
        self.root.add_child(longest)
        
        for i in range(1,len(text)):
            longest = self.__expand_suffixes(longest, text, i)
            
        
    def check_text(self, text):
        if text[-1] != end_of_string:
            return text + end_of_string
        return text
    
    def __expand_suffixes(self, longest ,text, cur_pos):
        
        current = longest
        previous = None
        
        while not current.is_child(text[cur_pos]):
            
            child = self.Trie_Node(text[cur_pos], current)
            current.add_child(child)
            # print(current.children)
            # print(current is self.root)

            
            if previous is not None:
                previous.link = child
                
            current = current.link
            previous = child
            
        if current is self.root and self.root.get_child(text[cur_pos]) is previous:
            previous.link = self.root
        else:
            previous.link = current.get_child(text[cur_pos])
            
        return longest.get_child(text[cur_pos])
        
        
            
    def is_in_text(self, text):
        
        pos = 0
        node = self.root

        while node.is_child(text[pos]):

            node = node.get_child(text[pos])
            pos += 1
            if pos == len(text):
                return True
            
        return False
    
    def print_Trie(self):
        self.__printRecur(self.root)
            
    def __printRecur(self, node):
        print(node.children)
        for x in node.children.values():

            self.__printRecur(x)
            


In [221]:
class Link_Trie:
    class Trie_Node:
        def __init__(self, letter, parent, children = None, link = None, depth = 0):
            self.letter = letter
            self.parent = parent
            self.depth = depth
            
            self.link = link
            
            if children is None:
                self.children = dict()
            else:
                self.children = children
                
        def add_child(self, child):

            self.children[child.letter] = child
            
        def is_child(self,letter):
            return letter in self.children.keys()
        
        def get_child(self, letter):
            return self.children[letter]
        
            
    def __init__(self, text):
        
        self.root = self.Trie_Node(None, None)
        self.root.link = self.root
        
        text = self.check_text(text)
        
        head = self.root
        
        for i in range(0,len(text)):
            
            leaf = self.__add_suffix(text, i, head)
            head = self.__up_link_down(leaf)
                        
        
    def check_text(self, text):
        if text[-1] != end_of_string:
            return text + end_of_string
        return text
    
    def __add_suffix(self, text ,pos, node):
        
        curr = node
        
        for i in range(pos + node.depth, len(text)):
            
            child = self.Trie_Node(text[i], curr, depth = curr.depth+1)
            curr.add_child(child)
            curr = child
            
        return curr
    
    def __up_link_down(self, leaf):
        
        curr = leaf
        queue = []
        
        
        
        while curr.link is None:
            queue.append(curr.letter)
            curr = curr.parent
            
        node = curr.link
        
        if curr is self.root:
            char = queue.pop()
            curr = curr.get_child(char)
            curr.link = self.root
        
        while (queue):
            char = queue.pop()
            if (node.is_child(char)):
                node = node.get_child(char)
                curr = curr.get_child(char)
                curr.link = node
            else:
                break
        return node
        
        
            
    def is_in_text(self, text):
        
        pos = 0
        node = self.root

        while node.is_child(text[pos]):

            node = node.get_child(text[pos])
            pos += 1
            if pos == len(text):
                return True
            
        return False
    
    def print_Trie(self):
        self.__printRecur(self.root)
            
    def __printRecur(self, node):
        print(node.children)
        for x in node.children.values():

            self.__printRecur(x)
            


In [224]:
x = Trie("mama")
x.print_Trie()
x.is_in_text("ma")

{'m': <__main__.Trie.Trie_Node object at 0x000001AB0080DB38>, 'a': <__main__.Trie.Trie_Node object at 0x000001AB0080DBA8>, 'ဦ': <__main__.Trie.Trie_Node object at 0x000001AB0100CAC8>}
{'a': <__main__.Trie.Trie_Node object at 0x000001AB0080D470>}
{'m': <__main__.Trie.Trie_Node object at 0x000001AB0100CA20>, 'ဦ': <__main__.Trie.Trie_Node object at 0x000001AB0100CB00>}
{'a': <__main__.Trie.Trie_Node object at 0x000001AB0100C2E8>}
{'ဦ': <__main__.Trie.Trie_Node object at 0x000001AB0100C908>}
{}
{}
{'m': <__main__.Trie.Trie_Node object at 0x000001AB0100C668>, 'ဦ': <__main__.Trie.Trie_Node object at 0x000001AB0100C588>}
{'a': <__main__.Trie.Trie_Node object at 0x000001AB0100C320>}
{'ဦ': <__main__.Trie.Trie_Node object at 0x000001AB0100CC88>}
{}
{}
{}


True

In [225]:
x = Link_Trie("mama")
x.print_Trie()


{'m': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100C390>, 'a': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100C4A8>, 'ဦ': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100C080>}
{'a': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100CE80>}
{'m': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100CE10>, 'ဦ': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100CC50>}
{'a': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100C630>}
{'ဦ': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100CBA8>}
{}
{}
{'m': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100C518>, 'ဦ': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100C470>}
{'a': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100CF98>}
{'ဦ': <__main__.Link_Trie.Trie_Node object at 0x000001AB0100C8D0>}
{}
{}
{}


In [ ]:
class Mc_Creight_Suffix_Tree:
    def init(self):
        